Neural networks can be constructed using the torch.nn package.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 1. Definition of Model
class Net(nn.Module):
    def __init__(self):
        
        super(Net, self).__init__()
        
        # 1 input image channel, 6 output channels, 3x3 square convolution
        self.conv1 = nn.Conv2d(1,6,3)
        # 6 input image channel, 16 output channels, 3x3 square convolution
        self.conv2 = nn.Conv2d(6,16,3)
        
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        # 第一维是个数
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [17]:
# 2. processing inputs and calling backward
params = list(net.parameters())
print(len(params))
# 每一个Conv2d, max_pool2d, relu, Linear 都占一个
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [18]:
#　Only for test
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)
net.zero_grad()
out.backward(torch.randn(1, 10))


tensor([[-0.0484, -0.1890, -0.0749, -0.0192, -0.0432,  0.0675, -0.0446, -0.0539,
         -0.0896, -0.1431]], grad_fn=<ThAddmmBackward>)


torch.nn only supports mini-batches. The entire torch.nn package only supports inputs that are a mini-batch of samples, and not a single sample.

For example, nn.Conv2d will take in a 4D Tensor of **nSamples x nChannels x Height x Width**.

If you have a single sample, just use input.unsqueeze(0) to add a fake batch dimension.

autograd.Function - Implements forward and backward definitions of an autograd operation. 
**Every Tensor operation creates at least a single Function node that connects to functions that created a Tensor and encodes its history.**

In [21]:
# 3. Loss
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.8151, grad_fn=<MseLossBackward>)


In [23]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [29]:
print(net.conv1.bias.grad)

tensor([-0.0190,  0.0032,  0.0093, -0.0011,  0.0003,  0.0036])


In [28]:
net.zero_grad()
loss.backward()

In [30]:
# 4. Update the weights
# weight = weight - learning_rate * gradient
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [31]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update
